In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
df = pd.read_csv(
    '/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',
    encoding='ISO-8859-1',
    header=None
)
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df.dtypes

target     int64
ids        int64
date      object
flag      object
user      object
text      object
dtype: object

In [4]:
for col in df.columns :
    print("#####################")
    print( df[col].value_counts())

#####################
target
0    800000
4    800000
Name: count, dtype: int64
#####################
ids
1972193428    2
1989776729    2
1989776908    2
1564543229    2
1690832844    2
             ..
2197310477    1
2197310452    1
2197310381    1
2197310235    1
2197311763    1
Name: count, Length: 1598315, dtype: int64
#####################
date
Mon Jun 15 12:53:14 PDT 2009    20
Fri May 22 05:10:17 PDT 2009    17
Mon Jun 15 13:39:50 PDT 2009    17
Fri May 29 13:40:04 PDT 2009    17
Fri Jun 05 14:13:07 PDT 2009    16
                                ..
Tue Jun 16 08:34:52 PDT 2009     1
Mon Jun 15 14:18:18 PDT 2009     1
Mon Jun 15 14:18:19 PDT 2009     1
Mon Jun 15 14:18:21 PDT 2009     1
Mon Jun 15 14:18:23 PDT 2009     1
Name: count, Length: 774363, dtype: int64
#####################
flag
NO_QUERY    1600000
Name: count, dtype: int64
#####################
user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
       

In [5]:
for col in df.columns :
    print("#####################")
    print(col, ":", df[col].nunique())

#####################
target : 2
#####################
ids : 1598315
#####################
date : 774363
#####################
flag : 1
#####################
user : 659775
#####################
text : 1581466


In [6]:
df[['dow', 'month', 'day', 'time', 'tz', 'year']] = df['date'].str.split(expand=True)
df['day'] = df['day'].astype(int)
df['year'] = df['year'].astype(int)

df.drop(columns=["flag", "date"], inplace=True)

In [7]:
df.head()

,target,ids,user,text,dow,month,day,time,tz,year
0,0,1467810369,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon,Apr,6,22:19:45,PDT,2009
1,0,1467810672,scotthamilton,is upset that he can't update his Facebook by ...,Mon,Apr,6,22:19:49,PDT,2009
2,0,1467810917,mattycus,@Kenichan I dived many times for the ball. Man...,Mon,Apr,6,22:19:53,PDT,2009
3,0,1467811184,ElleCTF,my whole body feels itchy and like its on fire,Mon,Apr,6,22:19:57,PDT,2009
4,0,1467811193,Karoli,"@nationwideclass no, it's not behaving at all....",Mon,Apr,6,22:19:57,PDT,2009


In [8]:
df.dtypes

target     int64
ids        int64
user      object
text      object
dow       object
month     object
day        int64
time      object
tz        object
year       int64
dtype: object

In [9]:

df['user'] = df['user'].astype('string')
df['text'] = df['text'].astype('string')
df['dow'] = df['dow'].astype('category')
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time
df['tz'] = df['tz'].astype('category')


month_map = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}
df['month'] = df['month'].map(month_map)




In [10]:
df.dtypes

target             int64
ids                int64
user      string[python]
text      string[python]
dow             category
month              int64
day                int64
time              object
tz              category
year               int64
dtype: object

In [11]:
type(df['time'][0])


datetime.time